In [1]:
# does not work 
from envs.linear_bandit import ret_feature_func
import numpy as np
def sigmoid(z):
    return 1/(1 + np.exp(-z))

cur_state = np.array([1]) 

def get_p(action_one,action_two):
    feature_func = ret_feature_func(num_action=len(actions), state_dim=1)
    feature_one = feature_func(cur_state, action_one) #TODO 现在这个feature_fun是单调的
    feature_two = feature_func(cur_state, action_two)
    feature = np.concatenate([feature_one, feature_two])
    # score_param = np.array([1.0, 2.0, 3.0, 4.0], np.float32)
    score_param = np.array([1.0, 1.0, -1, -1], np.float32)
    score = np.dot(feature, score_param)
    p = sigmoid(score)
    return p

for i in range(len(actions)):
    for j in range(len(actions)):
        action_one = actions[i]
        action_two = actions[j]
        print(i,j,'  ', get_p(action_one,action_two))

NameError: name 'actions' is not defined

# based on qqg's paper

In [144]:
import torch
import torch.nn as nn
actions = [-10,0,10]
cur_state = np.array([0]) 

class NonMonotonicScalarToVectorNN(nn.Module):
    def __init__(self):
        super(NonMonotonicScalarToVectorNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)     # Input layer (1 -> 10)
        self.fc2 = nn.Linear(10, 20)    # Hidden layer (10 -> 20)
        self.fc3 = nn.Linear(20, 2)     # Output layer (20 -> 2)      
    def forward(self, x):
        x = torch.tanh(self.fc1(x))      # Using sin activation for non-monotonicity
        x = torch.tanh(self.fc2(x))     # Tanh activation adds more non-linearity
        x = self.fc3(x)                 # Output layer
        return x
        
feature_func = NonMonotonicScalarToVectorNN()
def get_p(action_one,action_two):
    feature_one = feature_func(torch.tensor([[action_one]], dtype=torch.float))[0].detach().numpy()
    feature_two = feature_func(torch.tensor([[action_two]], dtype=torch.float))[0].detach().numpy()
    score_param =  np.array([[0.0, -1.0],[1, 0]], np.float32) 
    score = feature_one@score_param@feature_two
    p = sigmoid(score)
    return p

p_list = np.zeros([len(actions),len(actions)])
for i in range(len(actions)):
    for j in range(len(actions)):
        action_one = actions[i]
        action_two = actions[j]
        p = get_p(action_one,action_two)
        if i==j:
            assert p==0.5
        p_list[i][j] = p
        print(actions[i],actions[j],'  ',p )


# check transition
def check_cyclic_order(matrix):
    items = [str(i) for i in actions]
    # Generate all permutations of the items
    for perm in itertools.permutations(items):
        # Extract preferences based on the current permutation
        if (matrix[items.index(perm[0])][items.index(perm[1])] > 0.5 and
            matrix[items.index(perm[1])][items.index(perm[2])] > 0.5 and
            matrix[items.index(perm[2])][items.index(perm[0])] > 0.5):
            return True, perm  # Found a cyclic order
    return False, None  # No cyclic order found
check_cyclic_order(p_list)

-10 -10    0.5
-10 0    0.5121860832312276
-10 10    0.49168890282075806
0 -10    0.4878139100393991
0 0    0.5
0 10    0.5106681920667224
10 -10    0.5083110991118743
10 0    0.4893317885227229
10 10    0.5


(True, ('-10', '0', '10'))